# 1. A brief overview of TensorBoard
* TensorBoard là một web application dùng để hình dung các metric, parameter và structure của deep neural network dc tạo ra bằng TensorFlow.
* Nó giúp ta dễ dàng debug và tối ưu hóa deep neural network một cách nhanh hơn và dễ dàng hơn.
* Với TensorBoard ta có thể dễ dàng so sánh giữa các network với nhau. Điều này cho phép ta có thể hiệu chỉnh các hyperparameter, thay đổi network architecture và sau đó đánh giá lại network để xem nó có cải thiện hơn không so với các network trước đó. Ngoài ra, nó cũng cho phép hiệu chỉnh trên từng epoch giúp ta sớm loại bỏ các lần đào tạo network mà không tốt ngay lập tức từ đó tiết kiệm thời gian và tiền bạc. Có thể tìm hiểu thêm tại [https://www.tensorflow.org/guide](https://www.tensorflow.org/guide).

# 2. Setting up TensorBoard
## 2.1. Installing TensorBoard
* Khi cài TensorFlow thì mặc định TensorBoard sẽ được cài kèm theo, tuy nhiên nếu ta muốn update TensorBoard thì sài lệnh sau:
  ```shell
  pip install -U tensorboard
  ```

## 2.2. How TensorBoard talks to Keras/TensorFlow
* TensorBoard và TensorFlow chia sẻ thông tin lẫn nhau trên một ***log directory*** [thư mục nhật kí]. Khi TensorFlow/Keras đào tạo network, chúng đồng thời ghi các ***metric*** [số liệu] và ***activation histogram*** [biểu đồ hoạt hóa] vào log directory mà ta chỉ định. Ta có thể dùng mã dưới đây để tạo ra log directory:
  ```shell
  mkdir ./tb_log
  ```

## 2.3. Running TensorBoard
* Chúng ta có thể start TensorBoard bằng lệnh dưới đây:
  ```shell
  tensorboard --logdir ./tb_log --port 6006
  ```
  * Ở đây `--log_dir` chỉ định đường dẫn log directory.
  * `--port` chỉ định PORT
* Bây giờ khi truy cập vào địa chỉ [http://localhost:6006](http://localhost:6006) ta nhận được kết quả như sau:<br>
  
  <center>

    ![](./images/03.00.png)

  </center>

* Lúc này do ta chưa có bất kì một network nào nên nó trống trơn vậy thôi, bình tĩnh.

# 3. Connecting Keras to TensorBoard
* Sau khi start TensorBoard, tất cả những gì còn lại là yêu cầu TensorBoard ghi nhật kí vào log directory mà ta chỉ định (ở đây là `tb_log`). Để thực hiện các điều vừa nêu trên ta dùng một lớp gọi là **Keras Callbacks**.

## 3.1. Introducing Keras callbacks
* **Callbacks** trong Keras là những hàm **có thể chạy trong quá trình đào tạo network**. Nó có thể thực hiện những hành động như: 
  * Lưu các network weights sau mỗi epoch.
  * Ghi lại những ghi chú do user định nghĩa như I love you chẳng hạn.
  * Thay đổi các hyperparameter.
  * Ghi các thông tin cần thiết của network vào log directory.
  * Thậm chí có thể custom các callback luôn.
* Màu nhiệm lắm, tìm hiểu thêm tại [https://keras.io/api/callbacks](https://keras.io/api/callbacks/).
* Các callbacks là các object cần được cấu hình trước khi đào tạo model. Chúng ta có thể tạo một `list` các callback với những thông tin mà chúng ta cần ghi hoặc sửa đổi trong quá trình đào tạo network. Sau đó bỏ `list` này vào như là một đối số trong hàm `fit()` của model. Cuối cùng các callback này sẽ tự động thực hiện các nhiệm vụ mà ta đã định nghĩa ở cuối mỗi epoch.

## 3.2. Creating a TensorBoard callback
* Hãy lấy lại ví dụ MLP model cho wine quantity mà ta đã làm ở Chapter 02, chúng ta sẽ thêm các callback object vào quá trình đào tạo model.
* Trước tiên, chúng ta sẽ tạo hàm `create_callback` và nó trả về một `list` các object callback để ta truyền vào hàm `fit()` như sau:

In [1]:
from keras.callbacks import TensorBoard

In [2]:
def create_callbacks():
    tensorboard_callback = TensorBoard(log_dir="./tb_log/mlp",
                                       histogram_freq=1,
                                       batch_size=32, write_graph=True, write_grads=False)
    
    return [tensorboard_callback]

* Hãy tìm hiểu các tham số của hàm trên:
  * `log_dir`: Đây là đường dẫn mà chúng ta ghi các log file cho TensorBoard.
  * `histogram_freq`: Dùng để chỉ định tần suất tính toán các biểu đồ histograms cho các activation function và weight qua các epoch. Nó mặc định là 0 tức nó sẽ không tạo ra histogram.
  * `batch_size`: Số lượng observation dùng để tính toán histogram, mặc định là 32.
  * `write_graph`: Mang giá trị Boolean, cho TensorBoard biết để trực quan network graph.
  * `write_grads`: Mang giá trụ Boolean, cho TensorBoard biết để tính toán histogram của gradient.
* Bây giờ chúng ta sẽ gọi lại các hàm MLP model của bài toán wine quality ở Chapter 02.

In [4]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
from keras.layers import Input, Dense
from keras.models import Model
import matplotlib.pyplot as plt
import seaborn as sns

TRAIN_DATA = "./data/train/train_data.csv"
VAL_DATA = "./data/val/val_data.csv"
TEST_DATA = "./data/test/test_data.csv"

def load_data():
    '''Load train, val and test datasets from disk.'''
    train = pd.read_csv(TRAIN_DATA)
    val = pd.read_csv(VAL_DATA)
    test = pd.read_csv(TEST_DATA)
    
    '''Using sklearn's StandardScaler to scale our data to 0 mean and unit variance.'''
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    val = scaler.transform(val)
    test = scaler.transform(test)
    
    '''We will use a dict to keep all this data.'''
    data = dict()
    data['train_y'] = train[:, 10]
    data['train_X'] = train[:, 0:9]
    data['val_y'] = val[:, 10]
    data['val_X'] = val[:, 0:9]
    data['test_y'] = test[:, 10]
    data['test_X'] = test[:, 0:9]
    
    '''Keep the `scaler` so we can unscale prediction in the future.'''
    data['scaler'] = scaler
    
    return data

def build_network(input_features=None):
    inputs = Input(shape=(input_features,), name='input') # input layer
    x = Dense(32, activation='relu', name='hidden')(inputs) # hidden layer với 32 neuron
    prediction = Dense(1, activation='linear', name='final')(x) # output layer với 1 neuron
    
    model = Model(inputs=inputs, outputs=prediction) # build model với input và output
    model.compile(optimizer='adam', loss='mean_absolute_error') # biên dịch model với adam optimizer và loss mà MAE
    
    return model

* Tạo callback object

In [5]:
callbacks = create_callbacks()

* Còn đây là các bước tương tự như đào tạo model ở Chapter 02.

In [6]:
data = load_data()
input_features = data['train_X'].shape[1]
model = build_network(input_features=input_features)

In model summary.

In [8]:
print("Training data shape: {}".format(data['train_X'].shape))
model.summary()

Training data shape: (3918, 9)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 9)]               0         
_________________________________________________________________
hidden (Dense)               (None, 32)                320       
_________________________________________________________________
final (Dense)                (None, 1)                 33        
Total params: 353
Trainable params: 353
Non-trainable params: 0
_________________________________________________________________


* Fit model với callbacks object.

In [9]:
model.fit(x=data['train_X'], y=data['train_y'],
          batch_size=32, epochs=200, verbose=1, 
          validation_data=(data['val_X'], data['val_y']),
          callbacks=callbacks)

'''Save model'''
model.save("./models/wine_quality/regression_model")

Epoch 1/200
123/123 [==============================] - 1s 6ms/step - loss: 0.5760 - val_loss: 0.4747
Epoch 2/200
123/123 [==============================] - 0s 3ms/step - loss: 0.4188 - val_loss: 0.3559
Epoch 3/200
123/123 [==============================] - 0s 2ms/step - loss: 0.3320 - val_loss: 0.2981
Epoch 4/200
123/123 [==============================] - 0s 3ms/step - loss: 0.2785 - val_loss: 0.2587
Epoch 5/200
123/123 [==============================] - 0s 3ms/step - loss: 0.2502 - val_loss: 0.2455
Epoch 6/200
123/123 [==============================] - 0s 4ms/step - loss: 0.2365 - val_loss: 0.2457
Epoch 7/200
123/123 [==============================] - 0s 3ms/step - loss: 0.2303 - val_loss: 0.2374
Epoch 8/200
123/123 [==============================] - 0s 3ms/step - loss: 0.2257 - val_loss: 0.2385
Epoch 9/200
123/123 [==============================] - 0s 3ms/step - loss: 0.2223 - val_loss: 0.2377
Epoch 10/200
123/123 [==============================] - 0s 4ms/step - loss: 0.2203 - val_lo

<hr>

* Chúng ta sẽ tạo hàm `create_callbacks_5hidden` cho model deep neural network với 5 hidden layer.

In [11]:
def create_callbacks_5hidden():
    tensorboard_callback = TensorBoard(log_dir="./tb_log/dnn", histogram_freq=1,
                                       batch_size=32, write_graph=True, write_grads=False)
    
    return [tensorboard_callback]

* Định nghĩa hàm `build_network_5hidden`

In [12]:
def build_network_5hidden(input_features=None):
    # first we specify an input layer, with a shape == features
    inputs = Input(shape=(input_features,), name="input")
    x = Dense(32, activation='relu', name="hidden1")(inputs)
    x = Dense(32, activation='relu', name="hidden2")(x)
    x = Dense(32, activation='relu', name="hidden3")(x)
    x = Dense(32, activation='relu', name="hidden4")(x)
    x = Dense(16, activation='relu', name="hidden5")(x)
    # for regression we will use a single neuron with linear (no) activation
    prediction = Dense(1, activation='linear', name="final")(x)

    model = Model(inputs=inputs, outputs=prediction)
    model.compile(optimizer='adam', loss='mean_absolute_error')
    
    return model

* Đào tạo `model_5hidden` với `callbacks_5hidden`.

In [13]:
callbacks_5hidden = create_callbacks_5hidden()
model_5hidden = build_network_5hidden(input_features=input_features)
model_5hidden.fit(x=data["train_X"], y=data["train_y"], 
                  batch_size=32, epochs=500, verbose=1, 
                  validation_data=(data["val_X"], data["val_y"]),
                  callbacks=callbacks_5hidden)

'''Save model'''
model_5hidden.save("./models/wine_quality_5hidden/regression_model")

Epoch 1/500
123/123 [==============================] - 1s 6ms/step - loss: 0.5339 - val_loss: 0.3645
Epoch 2/500
123/123 [==============================] - 1s 5ms/step - loss: 0.3076 - val_loss: 0.2855
Epoch 3/500
123/123 [==============================] - 0s 4ms/step - loss: 0.2583 - val_loss: 0.2565
Epoch 4/500
123/123 [==============================] - 0s 4ms/step - loss: 0.2393 - val_loss: 0.2365
Epoch 5/500
123/123 [==============================] - 1s 4ms/step - loss: 0.2271 - val_loss: 0.2335
Epoch 6/500
123/123 [==============================] - 1s 4ms/step - loss: 0.2202 - val_loss: 0.2338
Epoch 7/500
123/123 [==============================] - 1s 4ms/step - loss: 0.2174 - val_loss: 0.2266
Epoch 8/500
123/123 [==============================] - 0s 4ms/step - loss: 0.2100 - val_loss: 0.2268
Epoch 9/500
123/123 [==============================] - 1s 4ms/step - loss: 0.2104 - val_loss: 0.2285
Epoch 10/500
123/123 [==============================] - 0s 4ms/step - loss: 0.2048 - val_lo

# 4. Using TensorBoard
## 4.1. Visualizing training
* Bây giờ khi ta truy cập vào đường dẫn [http://localhost:6006](http://localhost:6006) và nhận kết quả như sau:
  <center>

    ![](./images/03.01.png)

  </center>

> **Nhận xét**
> * Ở plot 1, nó hiển thị giá trị của $\mathrm{MAE}$ loss function trên trục $y$ qua từng epoch đại diện bởi trục $x$. Chúng ta có thể biết đường nào đại diện cho model nào dựa vào chú thích của TensorBoard ở góc bottom-left của hình trên.
> * Hình hai thể hiện giá trị của $\mathrm{MAE}$ loss function qua từng iteration. Cụ thể ta có `batch_size` là 32, training data có 3918 observation, MLP's epoch = 200 và DNN's epoch = 500 $\Rightarrow$ ta có tổng cộng lần lượt số iteration cho MLP và DNN là $\dfrac{3918}{32} \times 200 \approx 24488$ và $\dfrac{3918}{32} \times 500 \approx 62219$. 
> * Nhìn vào biểu đồ 1 ta thấy, loss value của training data và val data của MLP cao nhưng không bị overfitting. Còn với DNN, loss value của training data tuy thấp nhưng trên val data thì quá cao, tách biệt lớn vậy DNN bị overfitting.

## 4.2. Visualizing network graphs
## 4.3. Visualizaing a broken network